# Transforming quantized model prepared for running on FPGA

## Converting QONNX format to FINN-QONNX format

QONNX format is from the Brevitas library. To run transformation through FINN, we need to convert the model to a FINN-QONNX format.

Whenever we want to instantiate an onnx model, we can use `ModelWrapper(filepath)`

### Load in the brevitas QONNX model

In [5]:
from qonnx.core.modelwrapper import ModelWrapper
from finn.util.visualization import showInNetron
#Path to the qonnx model exported by brevitas
brevitas_model_pth='27ml_rf/models/radio_27ml_export.onnx'

model=ModelWrapper(brevitas_model_pth)

!echo $VIVADO_PATH
!echo $HLS_PATH

/home/phu/Vivado/Vivado/2024.1


/home/phu/Vivado/Vitis_HLS/2024.1


### Visualizing the Brevitas ONNX model using `showInNetron`

In [ ]:
host_machine_ip='your host machine IP'
assert host_machine_ip!='your host machine IP', print('host_machine_ip not set')
showInNetron(brevitas_model_pth,localhost_url=host_machine_ip, port=8081)

Serving '27ml_rf/models/radio_27ml_export.onnx' at http://0.0.0.0:8081


### Converting the model to FINN-ONNX model

In [ ]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
model = model.transform(ConvertQONNXtoFINN())

finn_model_pth='27ml_rf/models/radio_27ml_finn.onnx'
model.save(finn_model_pth)

### Visualizing FINN-QONNX model. 
Notice how it is much easier to read compared to Brevitas ONNX model

In [8]:
showInNetron(finn_model_pth,localhost_url=host_machine_ip, port=8081)

Stopping http://0.0.0.0:8081
Serving '27ml_rf/models/radio_27ml_finn.onnx' at http://0.0.0.0:8081


## Network Surgery
From the graph above, you can see that the `input` goes through `MultiThreshold`, then `Add`, then `Conv` nodes. The problem we are facing is that FINN is unable to streamline the first `MultiThreshold` node. This is why we need to remove them manually.

Originally

`input`--> `MultiThreshold`--> `Add`--> `Conv` -->...

After network surgery:

`Add`--> `Conv` --> ...

However, during training, the input of `Add` node expects the output of `MultiThreshold`. Therefore, instead of passing through the raw data to our new model, we will manually perform what `MultiThreshold` does to our raw data (quantizing data) outside the model, then pass it through the new model as input.


Further information can be found in this finn's discussion: https://github.com/Xilinx/finn/discussions/420

### Tidying up the model
This step does not change parameters in the model, it is purely for reformatting the labels and renaming layers name for readability.

In [9]:
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.insert_topk import InsertTopK
from qonnx.transformation.general import (
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
)
import onnx
from qonnx.core.datatype import DataType

# tidy up
finn_model = ModelWrapper(finn_model_pth)

finn_model = finn_model.transform(InferShapes())
finn_model = finn_model.transform(InferDataTypes())
finn_model = finn_model.transform(GiveUniqueNodeNames())
finn_model = finn_model.transform(GiveReadableTensorNames())
finn_model.cleanup()

pre_net_surgery_pth='27ml_rf/models/radio_27ml_pre_nw_surgery.onnx'
finn_model.save(pre_net_surgery_pth)

showInNetron(pre_net_surgery_pth,localhost_url=host_machine_ip, port=8081)

Stopping http://0.0.0.0:8081
Serving '27ml_rf/models/radio_27ml_pre_nw_surgery.onnx' at http://0.0.0.0:8081


In [10]:
finn_model=ModelWrapper(pre_net_surgery_pth)

#Find the first 'Conv' node and store it in 'new_input_node'
new_input_node = finn_model.get_nodes_by_op_type("Conv")[0]   
#Find the input of that 'Conv' node (in this case it is the 'Add' node)
new_input_tensor = finn_model.get_tensor_valueinfo(new_input_node.input[0]) 

#Find the original input node of the model.
old_input_tensor = finn_model.graph.input[0] 

#Remove the old input node, and replace it with the new input tensor ('Add' node)
finn_model.graph.input.remove(old_input_tensor) 
finn_model.graph.input.append(new_input_tensor)

#Find the index of the new input node, and remove everything from index 0 to that index
#In this case, we will be removing index 0 and index 1, which are the 'inp' and 'MultiThreshold' nodes
#So now, the 'Add' node become the model input with index 0, and the 'Conv' node has index 1, and so on...
new_input_index = finn_model.get_node_index(new_input_node)
del finn_model.graph.node[0:new_input_index]

# postprocessing: remove final softmax node from training
# Removing any softmax node if there is any. This is because finn will use topK instead of softmax
# We have already removed the softmax node when building the model, so this can be ignored
softmax_node = finn_model.graph.node[-1]
softmax_in_tensor = finn_model.get_tensor_valueinfo(softmax_node.input[0])
softmax_out_tensor = finn_model.get_tensor_valueinfo(softmax_node.output[0])
finn_model.graph.output.remove(softmax_out_tensor)
finn_model.graph.output.append(softmax_in_tensor)
finn_model.graph.node.remove(softmax_node)

# remove redundant value_info for primary input/output
# othwerwise, newer FINN versions will not accept the model
if finn_model.graph.input[0] in finn_model.graph.value_info:
    finn_model.graph.value_info.remove(finn_model.graph.input[0])
if finn_model.graph.output[0] in finn_model.graph.value_info:
    finn_model.graph.value_info.remove(finn_model.graph.output[0])

# insert topK node in place of the final softmax node
# topK plays similar role to softmax
# k=1 means it pick only 1 classification with highest predictions value
finn_model = finn_model.transform(InsertTopK(k=1))

# manually set input datatype (not done by brevitas yet)
finnonnx_in_tensor_name = finn_model.graph.input[0].name
finnonnx_model_in_shape = finn_model.get_tensor_shape(finnonnx_in_tensor_name)
finn_model.set_tensor_datatype(finnonnx_in_tensor_name, DataType["INT8"])
print("Input tensor name: %s" % finnonnx_in_tensor_name)
print("Input tensor shape: %s" % str(finnonnx_model_in_shape))
print("Input tensor datatype: %s" % str(finn_model.get_tensor_datatype(finnonnx_in_tensor_name)))

# save modified model that is now ready for the FINN compiler
tidy_model_pth='27ml_rf/models/radio_27ml_tidy.onnx'
finn_model.save(tidy_model_pth)
print("Modified FINN-ready model saved to %s" % tidy_model_pth)

showInNetron(tidy_model_pth,localhost_url=host_machine_ip, port=8081)

Input tensor name: Add_0_out0
Input tensor shape: [1, 2, 1024]
Input tensor datatype: INT8
Modified FINN-ready model saved to 27ml_rf/models/radio_27ml_tidy.onnx
Stopping http://0.0.0.0:8081
Serving '27ml_rf/models/radio_27ml_tidy.onnx' at http://0.0.0.0:8081


# FINN Transformation
From here, we will setup a FINN's standard builder, with a few custom transformations, and export a bitfile which can be run using pynq on the FPGA.

Original version of the code below can be found here: [Original version](https://github.com/Xilinx/finn-examples/blob/main/build/vgg10-radioml/README.md)

Further information about setting up a builder for transformations can be found here: [Tutorial](https://github.com/Xilinx/finn/blob/main/notebooks/end2end_example/cybersecurity/3-build-accelerator-with-finn.ipynb)

### Defining custom steps for the builder
The builder has a few steps already prepared for us. However, since we are using a 1D conv layer, we will need to add 2 more custom steps to convert them from 1D to 2D

`step_pre_streamline` is for converting from our model from 3D tensors to 4D tensors. This is because we initially use 1D convolutional layers. This means the input shape will be changed from `1x2x1024` to `1x1x2x1024`

`step_convert_final_layers` is for converting the final layers (linear and topK) to hardware layers

Further discussion about 1D conversion to 2D: [Github Discussion](https://github.com/Xilinx/finn/discussions/418)

In [11]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors
from qonnx.transformation.general import GiveUniqueNodeNames

import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
import finn.transformation.streamline.absorb as absorb
from finn.builder.build_dataflow_config import DataflowBuildConfig
from finn.transformation.fpgadataflow.set_fifo_depths import InsertAndSetFIFODepths


def step_pre_streamline(model: ModelWrapper, cfg: DataflowBuildConfig):
    model = model.transform(Change3DTo4DTensors())
    model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
    return model


def step_convert_final_layers(model: ModelWrapper, cfg: DataflowBuildConfig):
    model = model.transform(to_hw.InferChannelwiseLinearLayer())
    model = model.transform(to_hw.InferLabelSelectLayer())
    model = model.transform(GiveUniqueNodeNames())
    return model

### Setting up Builder

In [ ]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
from finn.util.basic import alveo_default_platform

import os
import shutil
import os
import glob
import shutil
from pathlib import Path

"--------------------------------------------------------------------"
#Get the tidy.onnx model
model_name='radio_27ml_tidy'
model_file = tidy_model_pth

#Create a temporary folders (if none exist) to store intermediate transformations
finn_build_dir=os.getcwd()+'/tmp/'
os.environ["FINN_BUILD_DIR"]=finn_build_dir

final_output_dir="output/"
Path(finn_build_dir).mkdir(exist_ok=True)
Path(final_output_dir).mkdir(exist_ok=True)

#Remove all intermediate transformations from previous runs 
print(f'temp files will be built in {finn_build_dir}')
print(f'removing old temp files in {finn_build_dir}')
files = glob.glob(f'{finn_build_dir}*')
for f in files:
    if os.path.isdir(f):
        shutil.rmtree(f)
    elif os.path.isfile(f):
        os.remove(f)


print(f'output will be generated in {final_output_dir}')
"--------------------------------------------------------------------"

# which platforms to build the networks for
zynq_platforms = ["ZCU104"]
alveo_platforms = []
platforms_to_build = zynq_platforms + alveo_platforms


# determine which shell flow to use for a given platform
def platform_to_shell(platform):
    if platform in zynq_platforms:
        return build_cfg.ShellFlowType.VIVADO_ZYNQ
    elif platform in alveo_platforms:
        return build_cfg.ShellFlowType.VITIS_ALVEO
    else:
        raise Exception("Unknown platform, can't determine ShellFlowType")


# select target clock frequency
def select_clk_period(platform):
    return 4.0
    #return 5.0

from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType

# assemble build flow from custom and pre-existing steps
def select_build_steps(platform):
    return [
        "step_tidy_up",
        step_pre_streamline, #Custom steps above
        "step_streamline",
        "step_convert_to_hw",
        step_convert_final_layers,  #Custom steps above
        "step_create_dataflow_partition",
        "step_specialize_layers",
        "step_target_fps_parallelization",
        "step_apply_folding_config",
        "step_minimize_bit_width",
        "step_generate_estimate_reports",
        "step_hw_codegen",
        "step_hw_ipgen",
        "step_set_fifo_depths",
        "step_create_stitched_ip",
        "step_measure_rtlsim_performance",
        "step_out_of_context_synthesis",
        "step_synthesize_bitfile",
        "step_make_pynq_driver",
        "step_deployment_package",
    ]


# create a release dir, used for finn-examples release packaging
os.makedirs("release", exist_ok=True)

for platform_name in platforms_to_build:
    shell_flow_type = platform_to_shell(platform_name)
    if shell_flow_type == build_cfg.ShellFlowType.VITIS_ALVEO:
        vitis_platform = alveo_default_platform[platform_name]
        # for Alveo, use the Vitis platform name as the release name
        # e.g. xilinx_u250_xdma_201830_2
        release_platform_name = vitis_platform
    else:
        vitis_platform = None
        # for Zynq, use the board name as the release name
        # e.g. ZCU104
        release_platform_name = platform_name
    platform_dir = "release/%s" % release_platform_name
    os.makedirs(platform_dir, exist_ok=True)

    cfg = build_cfg.DataflowBuildConfig(
        steps=select_build_steps(platform_name),
        output_dir=final_output_dir+"output_%s_%s" % (model_name, release_platform_name),
        synth_clk_period_ns=select_clk_period(platform_name),
        target_fps=4600, #Target FPS, not guaranteed the model will achieve
        board=platform_name,
        shell_flow_type=shell_flow_type,
        vitis_platform=vitis_platform,
        split_large_fifos=True,
        standalone_thresholds=True,
        # enable extra performance optimizations (physopt)
        vitis_opt_strategy=build_cfg.VitisOptStrategyCfg.PERFORMANCE_BEST,
        generate_outputs=[
            build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
            build_cfg.DataflowOutputType.STITCHED_IP,
            # build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
            build_cfg.DataflowOutputType.BITFILE,
            build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
            build_cfg.DataflowOutputType.PYNQ_DRIVER,
        ],
        
    )
    build.build_dataflow_cfg(model_file, cfg)

    # copy bitfiles and runtime weights into release dir if found
    bitfile_gen_dir = cfg.output_dir + "/bitfile"
    files_to_check_and_copy = [
        "finn-accel.bit",
        "finn-accel.hwh",
        "finn-accel.xclbin",
    ]
    for f in files_to_check_and_copy:
        src_file = bitfile_gen_dir + "/" + f
        dst_file = platform_dir + "/" + f.replace("finn-accel", model_name)
        if os.path.isfile(src_file):
            shutil.copy(src_file, dst_file)

    weight_gen_dir = cfg.output_dir + "/driver/runtime_weights"
    weight_dst_dir = platform_dir + "/%s_runtime_weights" % model_name
    if os.path.isdir(weight_gen_dir):
        weight_files = os.listdir(weight_gen_dir)
        if weight_files:
            shutil.copytree(weight_gen_dir, weight_dst_dir)